https://towardsdatascience.com/covid-19-data-processing-58aaa3663f6

https://towardsdatascience.com/@bindiatwork

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
import plotly.express as px
import folium

print('Modules imported')



ModuleNotFoundError: No module named 'seaborn'

### Import dataset (data frame)

In [ ]:
deaths_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
deaths_df.head()

### Check shape of data

In [ ]:
deaths_df.shape

### Merging Confirmed, Deaths and Recovered
Before merging, we need to use melt() to unpivot DataFrames from current wide format into long format.

In [ ]:
deaths_df.columns[4:]

In [ ]:
dates = deaths_df.columns[4:]
dates

In [ ]:
death_df_long = deaths_df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Deaths'
)
death_df_long.head

### Data cleaning
Converting Date from string to datetime

Replacing missing value NaN

Coronavirus cases reported from 3 cruise ships should be treated differently

In [ ]:
death_df_long['Date'] = pd.to_datetime(death_df_long['Date'])
death_df_long

##### Count NaNs

In [ ]:
death_df_long.isna().sum()

In [ ]:
death_df_long['Province/State'] = death_df_long['Province/State'].fillna('None')
death_df_long

### Derived Tables

Tables made from other tables

In [ ]:
uk = death_df_long[death_df_long['Country/Region']=='United Kingdom']
uk

In [ ]:
us = death_df_long[death_df_long['Country/Region']=='US']
us

### Latest data

In [ ]:
full_latest = death_df_long[death_df_long['Date'] == max(death_df_long['Date'])].reset_index()
full_latest

In [ ]:
full_latest = death_df_long[death_df_long['Date'] == max(death_df_long['Date'])].reset_index()

In [ ]:
uk_latest = full_latest[full_latest['Country/Region']=='United Kingdom']
uk_latest

In [ ]:
uk_latest_grouped = uk_latest.groupby(['Country/Region', 'Province/State'])['Deaths'].sum().reset_index()

uk_latest_grouped.style.background_gradient(cmap='Pastel2_r')

### Group Data

In [ ]:
death_df_grouped = death_df_long.groupby('Country/Region')['Deaths'].sum().reset_index()
death_df_grouped

In [ ]:
us_deaths = death_df_grouped[death_df_grouped['Country/Region']=='US']
us_deaths

In [ ]:
uk_deaths = death_df_grouped[death_df_grouped['Country/Region']=='United Kingdom']
uk_deaths

In [ ]:
uk_deaths = death_df_grouped[death_df_grouped['Country/Region']=='United Kingdom']
uk_deaths

In [ ]:
uk_latest = full_latest[full_latest['Country/Region'] == 'United Kingdom']
print(uk_latest)

In [ ]:
uk_latest_grouped = uk_latest.groupby('Province/State')['Deaths'].sum().reset_index()
uk_latest_grouped

In [ ]:
total_all = death_df_long.groupby('Date')['Deaths'].sum().reset_index()
# get max date
total_all = total_all[total_all['Date']==max(total_all['Date'])].reset_index(drop=True)
print(total_all)

In [ ]:
# https://plot.ly/python/map-configuration/

geo_df = full_table.groupby(['Date', 'Country/Region'])['Confirmed', 'Deaths'].max()
geo_df = geo_df.reset_index()
geo_df['Date'] = pd.to_datetime(geo_df['Date'])
geo_df['Date'] = geo_df['Date'].dt.strftime('%d/%m/%Y')
geo_df['size'] = geo_df['Confirmed'].pow(0.3)


fig = px.scatter_geo(geo_df, locations="Country/Region", locationmode='country names', 
                     color="Confirmed", size='size', hover_name="Country/Region", 
                     range_color= [0, max(geo_df['Confirmed'])+2], 
                     projection="natural earth", animation_frame="Date", 
                     color_continuous_scale="Viridis",
                     title='Spread over time')
fig.update(layout_coloraxis_showscale=True)
fig.show()